# Lectura del corpus de entrenamiento

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/Curso IA - MeIA/datos/MeIA_2025_train.xlsx', sheet_name="MeIA_2025_train")
#Toma en cuenta que debes cambiar la ruta del archivo de tal modo que coincida con la ubicación
#de tu archivo
data['Polarity'] = data['Polarity'].astype(int) #para convertir las clases a tipo entero
data = data.drop(columns=['Town','Region','Type']) #solo si quieres quitar estas columnas
data = data.rename(columns={'Review': 'texto'}) #para estandarizar con los modelos de transfomers
data = data.rename(columns={'Polarity': 'labels'}) #para estandarizar con los modelos de transfomers
data['labels'] = data['labels'] - 1 #para que las clases estén de en el rango de [0-4]

print("Dimensiones del dataset de entrenamiento:", data.shape)
data.head()

Dimensiones del dataset de entrenamiento: (5000, 2)


,texto,labels
0,Un Restaurante te invita por su ambiente tan a...,1
1,Pagamos 25 pesos por la entrada y no es gran c...,2
2,Mi esposa y yo nos alojamos en el Dreams por 4...,2
3,"La única decepción puede no ser José Cuervo, p...",1
4,Cuando leí los comentarios sobre cómo son las ...,0


#Para abrir archivo de test

In [3]:
pf_test = pd.read_excel('/content/drive/MyDrive/Curso IA - MeIA/datos/MeIA_2025_test_wo_labels.xlsx', sheet_name="MeIA_2025_test_wo_labels")
pf_test['labels'] = data['labels']  # si 'Polarity' existe
pf_test = pf_test.rename(columns={'Review': 'texto'})#para estandarizar con los modelos de transfomers
pf_test = pf_test.drop(columns=['Town','Region','Type', 'ID'], errors='ignore')#solo si quieres quitar estas columnas
pf_test.head()

,texto,labels
0,Nos alojamos 5 noches en el Coco Beach Bungalo...,1
1,"Es lindo, pero me hubiera gustado que nos come...",2
2,"El lugar es muy bonito, lo tienen MUY descuida...",2
3,"Además de que Taxco es mágico, el zócalo tiene...",1
4,"Sin duda un lugar Mágico para visitar, debes i...",0


# diccionarios de tus clases

**----------------------------------------------MI CODIGO----------------------------------------------**

In [4]:
! nvidia-smi

Thu Jun 19 18:07:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
#!pip install datasets
#!pip install transformers==4.28.0
#!pip install --upgrade accelerate

In [6]:
import re

def limpiar_ruido(texto):
    # Pasa a minúsculas para detectar patrones
    texto = texto.lower()

    # Elimina fragmentos típicos de truncamiento
    texto = re.sub(r'ver\s*más', '', texto)
    texto = re.sub(r'leer\s*más', '', texto)
    texto = re.sub(r'https?://\S+', '', texto)  # Quitar URLs
    texto = re.sub(r'[^\w\sáéíóúüñ]', '', texto)  # Eliminar puntuación innecesaria


    # También puedes eliminar caracteres raros o múltiples espacios
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

In [7]:
data['texto'] = data['texto'].apply(limpiar_ruido)

In [8]:
data = data[data['texto'].str.strip() != '']
data = data.drop_duplicates(subset='texto')

In [9]:
from datasets import Dataset, DatasetDict

#Unicamente utilizamos el texto y las demás columnas se eliminan
print(data.head(10),"\n")

                                               texto  labels
0  un restaurante te invita por su ambiente tan a...       1
1  pagamos 25 pesos por la entrada y no es gran c...       2
2  mi esposa y yo nos alojamos en el dreams por 4...       2
3  la única decepción puede no ser josé cuervo pe...       1
4  cuando leí los comentarios sobre cómo son las ...       0
5  durante la cata compre una botella de tequila ...       0
6  la verdad muy costosa la entrada 500 pesos mex...       0
7  el lugar está decorado muy bonito tiene mesas ...       2
8  hace tiempo que fui al restaurante de este hot...       2
9  primero de todo el hotel está situado en una b...       1 



In [10]:
#from sklearn.model_selection import train_test_split

#train_df, eval_df = train_test_split(data, test_size=0.2, stratify=data['labels'], random_state=42)

#ds_train = Dataset.from_pandas(data, preserve_index=False)
#ds_test = Dataset.from_pandas(pf_test, preserve_index=False)

#dataset = DatasetDict({
    #'train': ds_train,
    #'test': Dataset.from_pandas(pf_test)
#})

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(data, test_size=0.2, stratify=data["labels"], random_state=42)

ds_train = Dataset.from_pandas(df_train, preserve_index=False)
ds_val = Dataset.from_pandas(df_val, preserve_index=False)

dataset = DatasetDict({
    'train': ds_train,
    'validation': ds_val
})


In [12]:
id2label = {0: "very negative", 1: "negative", 2: "neutral", 3: "positive", 4: "very positive"}
label2id = {"very negative": 0, "negative": 1, "neutral": 2, "positive": 3, "very positive": 4}

dataset

DatasetDict({
    train: Dataset({
        features: ['texto', 'labels'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['texto', 'labels'],
        num_rows: 1000
    })
})

In [13]:
#model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased" #BUENO (MEJOR DE MOMENTO)
#model_checkpoint = "finiteautomata/beto-sentiment-analysis" #BUENO
#model_checkpoint = "pysentimiento/bert-base-spanish-uncased-sentiment"
#model_checkpoint = "mrm8488/bert-spanish-cased-finetuned-sentiment"
#model_checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment" #BUENO
model_checkpoint = "VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis" #PROFESINAL - PROMETE MUCHO
#model_checkpoint = "ignacio-ave/beto-sentiment-analysis-spanish" #DE LO MEJOR

In [14]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
def tokenize_fn(examples):
    return tokenizer(
        examples["texto"],
        padding="max_length",  # o True si usas DataCollator
        truncation=True,
        max_length=128
    )

In [16]:
tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["texto"])
tokenized_dataset

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [17]:
tokenized_dataset["train"][1]

{'labels': 3,
 'input_ids': [4,
  1032,
  3860,
  1028,
  6354,
  26966,
  1040,
  1012,
  1012,
  1044,
  4408,
  1355,
  10248,
  1039,
  2809,
  1028,
  2993,
  1040,
  6391,
  1028,
  1044,
  1274,
  1646,
  1097,
  1987,
  1035,
  18477,
  30959,
  1207,
  5,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [18]:
from transformers import BertForSequenceClassification, BertTokenizerFast

model = BertForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
#!pip install evaluate

In [20]:
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from evaluate import load

# Se encarga del padding dinámico
data_collator = DataCollatorWithPadding(tokenizer)

accuracy_metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

In [21]:
from transformers import TrainingArguments

#training_args = TrainingArguments(
    #output_dir="bert-classifier",
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    #learning_rate=2e-5,
    #weight_decay=0.01,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    #num_train_epochs=3,
    #load_best_model_at_end=True,
    #remove_unused_columns=False,
    #metric_for_best_model="eval_accuracy",
#)
training_args = TrainingArguments(
    output_dir="bert-classifier",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    save_steps=200,
    logging_steps=50,
    logging_dir="./logs",
    learning_rate=3e-5,
    warmup_steps=200,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    load_best_model_at_end=True,
    remove_unused_columns=False,
    metric_for_best_model="eval_f1",
)

In [22]:
# Verifica el rango
print("Valores únicos de labels:", data['labels'].unique())
assert data['labels'].between(0, 4).all(), "Error: Hay etiquetas fuera del rango permitido (0-4)"

Valores únicos de labels: [1 2 0 3 4]


In [23]:
data['labels'].value_counts()

,count
labels,
4,1200
3,1100
2,1000
1,900
0,800


In [ ]:
from transformers import Trainer, EarlyStoppingCallback

ignore_mismatched_sizes=True

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [25]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jesus-torres6625 (jesus-torres6625-cucei) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,1.004400,1.158690,0.487000,0.456401,0.480718,0.487000
400,0.646400,1.194215,0.556000,0.555110,0.557601,0.556000
600,0.287500,1.482528,0.562000,0.563424,0.572114,0.562000
800,0.113500,1.838717,0.549000,0.553240,0.563993,0.549000
1000,0.058600,1.957301,0.560000,0.563074,0.569980,0.560000


TrainOutput(global_step=1000, training_loss=0.5361380369663239, metrics={'train_runtime': 759.9405, 'train_samples_per_second': 42.109, 'train_steps_per_second': 1.316, 'total_flos': 2104945139712000.0, 'train_loss': 0.5361380369663239, 'epoch': 8.0})

# Pasar test por el pipeline y escribir archivo de evaluación según el formato del MeIA

In [27]:
from transformers import pipeline

clf_pipe = pipeline(
    "text-classification",
    model=trainer.model,#Este modelo y tokenizador deben ser los que tú entrenaste
    tokenizer=tokenizer,
    device=0  # cambia a -1 si usas CPU
)

frases_test = pf_test['texto'].tolist();
batch = clf_pipe(frases_test, padding = True, truncation=True)

In [28]:
for res in batch:
    print(res)

{'label': 'positive', 'score': 0.9180054664611816}
{'label': 'positive', 'score': 0.6840594410896301}
{'label': 'neutral', 'score': 0.5797908306121826}
{'label': 'very positive', 'score': 0.9920663833618164}
{'label': 'positive', 'score': 0.9454929232597351}
{'label': 'positive', 'score': 0.8448989391326904}
{'label': 'positive', 'score': 0.9476206302642822}
{'label': 'positive', 'score': 0.9679901003837585}
{'label': 'positive', 'score': 0.8135083913803101}
{'label': 'positive', 'score': 0.9380396604537964}
{'label': 'neutral', 'score': 0.675925612449646}
{'label': 'very negative', 'score': 0.933363139629364}
{'label': 'positive', 'score': 0.5089864134788513}
{'label': 'negative', 'score': 0.8397213220596313}
{'label': 'very negative', 'score': 0.8188978433609009}
{'label': 'neutral', 'score': 0.9590418934822083}
{'label': 'positive', 'score': 0.7189210653305054}
{'label': 'positive', 'score': 0.9050280451774597}
{'label': 'neutral', 'score': 0.49426737427711487}
{'label': 'neutral', 

In [ ]:
escribir = open('/content/drive/MyDrive/Curso IA - MeIA/ChivaTeam-8.txt', 'w')#recuerda poner la dirección qué tu quieras

i = 0;
for res in batch:
    escribir.write(f"MeIA {i} {label2id[res['label']] + 1}\n")
    i+=1;
escribir.close()